# Tableau pour Cerlico

### Importation des modules

- pickle pour la récupération des données de Conscila
- pandas pour la gestion des csv et les calculs type excel
- numpy pour les calculs en général
- matplotlib pour les graphiques

In [1367]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy, datetime,pickle,re,codecs,unidecode
import io
from lxml import etree
from IPython.display import display

## Récupération des données précédentes

Le fichier *Timeline-RDVTACP.pkl* dans le répertoire *CorpusRugby2007* contient toutes les correspondances calculées prédédemment.

In [720]:
repRDVTACP="/Users/gilles/pCloud Drive/FOD/Copy/Recherche/Rugby/CorpusRugby2007/"
fRDVTACP=repRDVTACP+"Timeline-RDVTACP.pkl"

In [721]:
with open(fRDVTACP,"rb") as inFile:
    dfRDVTACP=pickle.load(inFile)

In [722]:
print ", ".join(dfRDVTACP.columns.tolist())

RJ1.00, RJ2.00, RJ3.00, RJ4.00, RJ5.00, RJ6.00, RJ7.00, RJ8.00, RC1.00, RC2.00, RC3.00, RC4.00, RC5.00, RC6.00, RC7.00, RC8.00, timeline, trscode, D1.00, D2.00, D3.00, D4.00, D5.00, D6.00, D7.00, D8.00, D9.00, D10.00, D11.00, D12.00, S1D1.00, S1D2.00, S1D3.00, S1D4.00, S1D5.00, S1D6.00, S1D7.00, S1D8.00, S1D9.00, S1D10.00, S1D11.00, S1D12.00, S2D1.00, S2D2.00, S2D3.00, S2D4.00, S2D5.00, S2D6.00, S2D7.00, S2D8.00, S2D9.00, S2D10.00, S2D11.00, S2D12.00, S3D1.00, S3D2.00, S3D3.00, S3D4.00, S3D5.00, S3D6.00, S3D7.00, S3D8.00, S3D9.00, S3D10.00, S3D11.00, S3D12.00, V1.00, V2.00, V3.00, V4.00, V5.00, V6.00, V7.00, V8.00, V9.00, V10.00, V11.00, V12.00, S1V1.00, S1V2.00, S1V3.00, S1V4.00, S1V5.00, S1V6.00, S1V7.00, S1V8.00, S1V9.00, S1V10.00, S1V11.00, S1V12.00, S2V1.00, S2V2.00, S2V3.00, S2V4.00, S2V5.00, S2V6.00, S2V7.00, S2V8.00, S2V9.00, S2V10.00, S2V11.00, S2V12.00, S3V1.00, S3V2.00, S3V3.00, S3V4.00, S3V5.00, S3V6.00, S3V7.00, S3V8.00, S3V9.00, S3V10.00, S3V11.00, S3V12.00, timecode, déb

Il faut choisir les colonnes à conserver pour le rythme, le débit, la vitesse

In [723]:
selectionColonnes="RJ5.00 RC5.00 timeline trscode D5.00 S1D5.00 S2D5.00 S3D5.00 V5.00 S1V5.00 S2V5.00 S3V5.00 timecode début fin numEvent événement numAct action commentaires numSpk Spk texte texte-PxP texte-colour texte-support dureeS1 dureeS2 PxP colour support sumPxP sumColour sumSupport pause suivant durée plage même".split(" ")
df=dfRDVTACP[selectionColonnes].copy()

## Ajustements sur les anciennes données

In [724]:
df["texte"]=df["texte"].str.strip()

In [1429]:
decMiTemps=1055
ajustements={
    u"apr\xe8s avoir demand\xe9 si tout le monde \xe9tait pr\xeat le coup d'envoi de la coupe du monde deux mille sept":u"apr\xe8s avoir demand\xe9 si tout le monde \xe9tait pr\xeat, le coup d'envoi de la coupe du monde deux mille sept",
    u'est donn\xe9 par David Skrela':u'est donn\xe9 par David Skrela.',
    u"spk2-071":u"oui il reviendra à l'en-avant de Mignoni",
    u"trotteuses que nous":u"trotteuses que nous regardez",
    u"spk1-1601":"",
    u"réagissent":u"réagissent on est bien",
    u"première gross alerte":u"première grosse alerte"
}
ajustementsTimes={
    1571.66:u"ce début de partie on est très fébriles côté français",
    1488.68:u"réagissent on est bien",
    1487.72:u"oh bien",
    1489.44:u"oh la la et qui vont",
    170.56:u"l'équipe oh hé bien",
    266.36:u"euh Bernard Laporte euh et oui pas content Bernard",
    693.2:u"féroce ah ouais",
    697.84:u"c'est un niveau",
    740.52:u"la défense française oh la rage",
    760.76:u"il est pris Rémy Martin",
    779.04:u"l'ascendant sur l'Argentine oui oui",
    857.24:u"ces plaquages ah c'est bon ça",
    858.92:u"de regroupement ah très bien",
    937.56:u"de la vitesse et oui oui",
    1073.68:u"oui ils sont devant nous",
    1079.36:u"Thierry hein",
    1152.48:u"va à droite il dirige",
    1302.6:u"attention de trouver des touches hein toujours trouver des touches explication Nani Corleto l'arrière de",
    1318.2:u"alors ça c'est intelligent ils progressent",
    1320.76:u"c'est intelligent le ballon est",
    1438.64:u"puisque les Français alors oui",
    1477.12:u"oh ça",
    1551.04:u"introduction argentine c'est plus dur",
    ## CF
    4462.82:u"oui donc euh déjà première consigne Bernard Laporte a dû dire soulagez Skrela",
    4516.98:u"cette fois pour faire jouer les gros au large oui au large oui c'est bien oui",
    4529.38:u"on se barre de ah ça y est hein ça y est mon cher Thierry on y est",
    4547.94:u"il a pas fait",
    4548.38:u"hé Rémi là ça ça s'arrête là hein il faut plus donner de ballons hein on les garde là il y a un Argentin au milieu",
    4553.06:u"ouais mais on leur dit tellement aussi qu'il on leur donne plus on donne plus",
    4559.62:u"de Rémi Martin l'intention est très bonne",
    4562.5:u"ou alors un problème de couleurs entre le bleu clair et bleu et bleu et bleu foncé",
    4595.74:u"désossé là Dominici regardez",
    4605.86:u"regardez le un il est en position de centre là argentin regardez vous le voyez à l'écran là",
    4609.58:u"et voilà il appelle d'ailleurs venez m'aider parce que la quand même ça va vite hé oui",
    4682.7:u"oui les airs",
    4687.86:u"dans le sol au sol",
    4727.9:u"un ballon qui est au fond ah oui",
    4773.78:u"avec ce ballon là qui est devant Harinordoquy caché par l'arbitre",
    4792.5:u"avec encore Thion marteau piqueur",
    4808.06:u"ah il y a des sécateurs hein vous l'avez dit ça regardez ça ça",
    4810.26:u"les tondeuses à gazon bloque au niveau des chevilles",
    4846.86:u"eh oui ce ballon gardé au il a gardé la balle au sol",
    4926.3:u"ouais Thierry",
    4929.02:u"l'homme a saignement",
    5028.86:u"et Hernandez",
    5075.14:u"hein voilà",
    5068.54:u"ouais voilà ce qu'il faut expliquer c'est quand y a un tas pour venir se joindre au tas",
    5178.18:u"qui fait ouais ouais",
    5227.3:u"espoir et même confiance à l'équipe de France ah enfin on occupe le terrain argentin",
    5240.3:u"oui et",
    5266.3:u"qui a pris l'intervalle allez allez",
    5289.78:u"du dynamisme c'est bien",
    5438.49:u"française et",
    5315.06:u"allez mêlée introduction française",
    5337.58:u"de l'équipe de France regardez voilà",
    5340.02:u"et on revoit les extérieurs",
    5447.82:u"en défense sur la la largeur du terrain parce qu'on a pas de ballon dynamique",
    5367.22:u"Jauzion ouais",
    5402.9:u"ah oui ah oui intense hein c'est formidable",
    5459.82:u"sa tête il pouvait pas le sortir Ostiglia ouais",
    5495.82:u"ah alors ça c'est terrible celle-là parce qu'en ouais",
    5503.02:u"ouh la la à dix-sept neuf",
    5504.18:u"on a huit points",
    5505.18:u"c'est terrible donc un essai transformé",
    5507.98:u"ça passe à droite ah oui",
    5542.78:u"pour Jauzion là-bas allez il faut aller dessus",
    5565.02:u"ah c'est parfait c'est parfait dans l'aire de jeu voilà parfait parfait",
    5680.02:u"carton jaune Thierry",
    5711.66:u"technique et volontaire c'est ça",
    5846.58:u"Betsen oh",
    ### CM
    5848.49:u"oh il avait très bien joué Chabal il est allé", #différent entre CF et CM
    5935.62:u"suppléer à David ouais ouais",
    5992.86:u"transmet oui oui",
    6001.82:u"Mignoni oh oh jaune jau(ne)",
    6071.54:u"je dois dire je vous l'accorde",
    6113.5:u"ouais",
    6143.1:u"de Corletto oui parce que",
    6159.22:u"et on la fait à l'endroit où on a l'habitude de la jouer",
    6203.78:u"bravo est impor(tante)",
    6226.7:u"dans le ah",
    6227.06:u"ah oui coeur",
    6230.7:u"sur ses ah non",
    6234.62:u"ça hein tout à fait",
    6245.7:u"avec un très bon pack d'avants et un Chabal royal",
    6271.54:u"Ledesma",
    6277:u"elle est pas très droite",
    6341.86:u"non non donnez pas donnez plus a dit l'arbitre",
    6370.42:u"pas lâché debout",
    6372.18:u"il l'a pas lâché",
    6373.7:u"on plaque en bas hein ça tombe hein",
    6417.46:u"ouais ouais moi y'a eu deux trois gifles qui sont parties sur ce coup-là",
    6421.7:u"ouais",
    6426.74:u"et on structure là il est au chaud et on structure",
    6428.54:u"le ballon est derrière le ballon est au chaud il peut rien nous arriver",
    6437.22:u"euh tranquilles avec ça absents ouais",
    6442.9:u"faudrait quand même euh",
    6473.78:u"(Spread)bury pénalité ah",
    6486.14:u"dès qu'on a le ballon c'est trop tard il faut regarder avant il a vu que le",
    6502.02:u"on est à la écoutez il reste",
    6540.58:u"dynamisme français",
    6541.14:np.NaN,
    6597.82:u"avec six points de s'il faut gagner",
    6600.78:u"comme ils craquent Fabien Landreau l'a dit",
    6602.26:u"tiens Elissalde à l'image allez",
    6605.7:u"continuez dans le rythme",
    6611.62:u"y'a du coaching",
    6677.66:u"c'est Martín Durand le troisième ligne qui rentre à la place d'Ostiglia",
    6681.46:u"et Senni- Sennilosa qui rentre à la place de Contepomi",
    6774.22:u"quand le numéro un à cinq est blessé on arrête le jeu quand c'est les autres numéros on arrête pas le jeu",
    6841.26:u"six ouais",
    6872.34:u"avec Senillosa",
    6891.94:u"voilà ouais bien sûr",
    6939.58:u"non c'est pas Elissalde qui va se charger de ce renvoi il va aller derrière ses avants derrière les",
    6959.18:u"ouais alors Harinordoquy oh",
    6960.9:u"et quelle belle prise de balle c'est bien",
    6966.42:u"allez y'a des gros là-bas ils vont",
    6980.1:u"mon* ballon qui bouge toujours",
    7004.54:u"qu'est-ce qu'on peut donner de ballons aujourd'hui à ces Argentins qui sont à l'agonie mais",
    7010.66:u"une seule leur courage leur courage leur agressivité leur courage",
    7130.82:u"appuis oui",
    7204.3:u"de France crucial hein",
    7208.62:u"ça c'est formidable voilà vivacité",
    7246.26:u"mais pas de bonus défensif dans une poule où y'a également l'Irlande",
    7257.38:u"avec Jauzion allez",
    7264.5:u"Elissalde non pas à gauche",
    7273.38:u"c'est contact oui mais on revient à la faute",
    7278.98:u"laissé aux allez",
    7290.5:u"mais ils tiennent ils tiennent les Argentins",
    7295.94:u"d'Argentine ils tiennent allez allez",
    7372.9:u"au goal average au point",
    7394.54:u"ici en quart de finale encore aujourd'hui",
    7415.94:u"l'Australie l'Australie oui",
    7441.46:u"et puis l'interception",
    7472.1:u"avec Christian Jean-Pierre Aubin Hueber",
    7479.78:u"nous on se retrouvera demain soir",
    7484.86:np.NaN,
}

In [1428]:
df[(df["début"].astype(float)>=(6420+decalage["CM"]))][["début","fin","texte","Spk"]]

,début,fin,texte,Spk
16606,7475.5,7478.22,l'ouverture des All Black la Nouvelle-Zélande ...,spk1
16607,7478.58,7479.42,à l'Italie,spk1
16608,7479.78,7480.94,nous on se retrouvera demain soir,spk1
16609,7481.34,7482.42,avec Jean-Michel Larqué,spk1
16610,7482.74,7483.9,pour un match de foot,spk1
16611,7484.1,7484.62,et quel match,spk1
16612,7484.86,7485.58,NaN,NaN
16613,7485.94,7486.94,entre l'Italie,spk1
16614,7487.1,7487.62,et la France,spk1
16615,7487.86,7489.14,très bonne fin de soirée,spk1


In [1322]:
print df.iloc[16548]["texte"]
print df.iloc[16550]["texte"]
print u"entre l'Italie"

c'est pas trop mal
c'est pas trop mal
entre l'Italie


In [1430]:
for ajustement in ajustements:
    df.loc[df['texte'] == ajustement,"texte"]=ajustements[ajustement]
for ajustementsTime in ajustementsTimes:
    df.loc[df["début"]==ajustementsTime,"texte"]=ajustementsTimes[ajustementsTime]

### Tests anciennes données

In [729]:
df[df["texte"]==u"euh Bernard Laporte euh"][["début","fin","texte"]]

,début,fin,texte


In [730]:
df.iloc[130:4207][["début","fin","texte","commentaires"]]

,début,fin,texte,commentaires
130,77.16,79.54,NaN,NaN
131,77.94,78.46,NaN,spk1-019
132,NaN,NaN,NaN,NaN
133,78.56,80.92,ah un ballon bien pris ah quoique relâché,
134,78.62,79.38,NaN,spk1-019
135,NaN,NaN,NaN,NaN
136,79.38,79.5,NaN,
137,79.7,79.94,NaN,spk1-020
138,NaN,NaN,NaN,NaN
139,80.02,81.74,NaN,


## Compilation des annotations de tours pour l'émotion, l'excitation et le type de discours

In [874]:
repAnnotations="/Users/gilles/ownCloud/Recherche/Rugby/"
fAnnotationsMP=repAnnotations+"France_Argentine-MP-0000.trs"
fAnnotationsCF=repAnnotations+"France_Argentine-CF-3250.trs"
fAnnotationsCM=repAnnotations+"France_Argentine-CM-4800.trs"
parser = etree.XMLParser(remove_blank_text=True)
maxTimeMP=1650.0
minTimeCM=4793.0
maxTimeCM=6500.0
minTimeCF=3250.0
maxTimeCF=4900.0

In [732]:
def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def jList(liste,sep=","):
    lListe=unique(liste)
    return sep.join([e for e in lListe if e!=""])

# Traitement pour une partie

In [1352]:
def getListeTours(idAnnotations):
    xmlTRS=etree.parse(fAnnotations[idAnnotations],parser)
    listeTours=[]
    for ligne in xmlTRS.xpath("//Turn"):
        debut=float(ligne.attrib["startTime"])
        if minTime[idAnnotations]<=debut<=maxTime[idAnnotations]:
            fin=float(ligne.attrib["endTime"])
            lTextes=[e.strip() for e in list(ligne.itertext()) if e.strip()!=""]
            lTour=[]
            lDiscours=[]
            lEmotions=[]
            lExcitation=[]
            for l in lTextes:
                splitL=[e.strip() for e in re.split(ur"\[([^\]]*)\]",l) if e!=""]
                lenSplitL=len(splitL)
                if lenSplitL%2!=0:
                    print "pas d'annotation", l, debut, fin
                    lTour.append(l.strip())
                else:
                    for tour,crochet in zip(*(iter(splitL),) * 2):
                        discours=""
                        emotion=""
                        excitation=""
                        lTour.append(tour.strip())
                        annotations=crochet.split(";")
                        if len(annotations)==3:
                            discours,emotion,excitation=annotations
                        elif len(annotations)==2:
                            discours,excitation=annotations
                        else:
                            print "problème d'annotation", annotations, debut, fin
                        lDiscours.append(discours)
                        lEmotions.append(emotion)
                        lExcitation.append(excitation)
            if lTour: 
                listeTours.append((debut,fin,jList(lTour,sep=" "),jList(lDiscours),jList(lEmotions),jList(lExcitation)))
    return listeTours

In [1353]:
def choisirAlignementsManuels(idAnnotations,dfAnnotations):
    for ix,row in dfAnnotations[dfAnnotations["début"]==""].iterrows():
        ixSelect=""
        ref=row["texte"]
        start=row["start"]+decalage[idAnnotations]
        debut=df.loc[(df["texte"]==ref),"début"].values
        debutPossible=df.loc[(df["texte"]==ref) & (abs(df["début"].astype(float)-start)<5),"début"].values
        if len(debutPossible)==1:
            possible=np.where(df.loc[(df["texte"]==ref),"début"].values==debutPossible[0])
            ixSelect=possible[0][0]+1
        xDebut=df[df["texte"]==ref].index
        if debut!=[]:
            print start, ref
            print debut
            print xDebut
            while not ixSelect:
                ixSelect=raw_input("Select index\n")
                try:
                    ixSelect=int(ixSelect)
                except:
                    ixSelect=""
            ixChoix.append(ixSelect)
            if ixSelect and ixSelect!="" and ixSelect<len(xDebut)+1:
                debut=df.iloc[xDebut[int(ixSelect)-1]]["début"]
                fin=df.iloc[xDebut[int(ixSelect)-1]]["fin"]    
                print ixSelect,debut,fin
                dfAnnotations.iloc[ix,dfAnnotations.columns.get_loc('début')]=debut
                dfAnnotations.iloc[ix,dfAnnotations.columns.get_loc('fin')]=fin
            else:
                print "mauvais choix"
            print
    return ixChoix

In [1355]:
def appliquerAlignementsManuels(idAnnotations,dfAnnotations):
    numChoix=0
    ixChoix=alignementsManuels[idAnnotations]
    print ixChoix
    for ix,row in dfAnnotations[dfAnnotations["début"]==""].iterrows():
        ref=row["texte"]
        start=row["start"]+decalage[idAnnotations]
        tDebut=df.loc[df["texte"]==ref,"début"].values
        xDebut=df[df["texte"]==ref].index
        if tDebut!=[]:
            ixSelect=ixChoix[numChoix]
            print start, ref, numChoix, ixSelect
            if ixSelect>len(xDebut)+1:
                print "erreur probable",numChoix
                print tDebut
            else:
                debut=df.iloc[xDebut[ixSelect-1]]["début"]
                fin=df.iloc[xDebut[ixSelect-1]]["fin"]
                if abs(start-debut)>1:
                    print "erreur probable",numChoix
                    print ixSelect,debut,fin, "(%f)"%(start-debut)
                    print tDebut
                dfAnnotations.iloc[ix,dfAnnotations.columns.get_loc('début')]=debut
                dfAnnotations.iloc[ix,dfAnnotations.columns.get_loc('fin')]=fin
            numChoix+=1    

In [1447]:
def traiterAnnotations(idAnnotations):
    
    listeTours=getListeTours(idAnnotations)
    
    colAnnotations=["start","end","texte","discours","émotion","excitation"]
    dfAnnotations=pd.DataFrame.from_records(listeTours,columns=colAnnotations)            
#    display(dfAnnotations[dfAnnotations["start"]>=5360.0])
    texteRef=[s.strip() for s in df[(df["texte"].notnull()) & (df["début"]>=minTime[idAnnotations]+decalage[idAnnotations]) & (df["début"]<=maxTime[idAnnotations]+decalage[idAnnotations])]["texte"].tolist()]
    texteRef=[l for l in texteRef if not l.startswith("spk")]
    texteAnnot=[s.strip()for s in dfAnnotations["texte"].tolist() if s!="" and s not in listeInsertions[idAnnotations]]

    dfAnnotations["début"]=""
    dfAnnotations["fin"]=""
    for t,ref,annot in [(i == j, i,j) for i, j in zip(texteRef, texteAnnot)]:
        if t:
            debut=df.loc[df["texte"]==ref,"début"].values
            fin=df.loc[df["texte"]==ref,"fin"].values
            if len(debut)==1:
                dfAnnotations.loc[dfAnnotations["texte"]==ref,"début"]=debut[0]
            if len(fin)==1:
                dfAnnotations.loc[dfAnnotations["texte"]==ref,"fin"]=fin[0]

    for insertion in listeInsertions[idAnnotations]:
        ixInsertion=dfAnnotations[dfAnnotations["texte"]==insertion].index
        dfAnnotations.loc[ixInsertion,"début"]=dfAnnotations.loc[ixInsertion,"start"]+decalage[idAnnotations]
        dfAnnotations.loc[ixInsertion,"fin"]=dfAnnotations.loc[ixInsertion,"end"]+decalage[idAnnotations]
    
    if alignementsManuels[idAnnotations]==[]:
        alignementsManuels[idAnnotations]=choisirAlignementsManuels(idAnnotations,dfAnnotations)
    else:
        appliquerAlignementsManuels(idAnnotations,dfAnnotations)
    return dfAnnotations


In [1449]:
repAnnotations="/Users/gilles/ownCloud/Recherche/Rugby/"
fAnnotations={
    "CM":repAnnotations+"France_Argentine-CM-4800.trs",
    "MP":repAnnotations+"France_Argentine-MP-0000.trs",
    "CF":repAnnotations+"France_Argentine-CF-3250.trs",
             }
minTime={
    "CM":4793.0,
    "MP":0.0,
    "CF":3250.0,
}
maxTime={
    "CM":6500.0,
    "MP":1620.0,
    "CF":4900.0
}
decalage={
    "CM":1055.0,
    "MP":0.0,
    "CF":1055.0,    
}
listeInsertions={
    "CM":[u"qui l'a fait commettre un en-avant",u"surclassés",u"ballons",u"de pénalités"],
    "MP":[],
    "CF":[u"qui l'a fait commettre un en-avant"],    
}
alignementsManuels={
    "CM":[14, 4, 6, 11, 1, 2, 2, 3, 1, 4,
          3, 4, 2, 1, 2, 2, 15, 16, 3, 2,
          12, 2, 6, 4, 5, 3, 3, 3, 1, 1, 2,
          17, 10, 4, 2, 2, 2, 2, 2, 1, 6,
          13, 1, 2, 14, 4, 1, 11, 7, 5, 2,
          8, 18, 2, 2, 7, 9, 2, 1, 2, 19, 1,
          8, 4, 1, 2, 1, 2, 7, 5, 2, 2, 9, 1,
          2, 10, 2, 4, 2, 11, 3, 4, 3, 1,
          2, 1, 2, 2, 2],
    "MP":[
          1,1,1,1,1,2,1,1,2,
          1,1,1,1,2,2,2,3,1,1,
          2,2,1,2,4,1,1,2,3,1,
          1,1,1,2,2,1,5,1,1,6,
          3,2,1,1,3,3
         ],
    "CF":[
          2,2,1,2,1,9,4,2,1,8,
          9,2,1,10,1,2,3,1,5,8,
          2,1,6,10,2,2,2,1,2,3,
          9,1,11,1,2,12,1,2,2,
          5,4,4,5,3,13,3,4,3,1,
          2,1,2,2,2,1,14,4,6
         ],        
}
partiesDF={}

In [1452]:
def calculerParties():
    for k in fAnnotations:
        partiesDF[k]=traiterAnnotations(k)

In [1453]:
calculerParties()

pas d'annotation oh hé bien 171.469 172.067
pas d'annotation euh Bernard Laporte euh 267.258 268.701
pas d'annotation oui 671.256 673.124
pas d'annotation ah ouais 694.107 694.65
pas d'annotation c'est un 698.763 699.06
pas d'annotation ouais on 712.739 713.255
pas d'annotation il est pris 761.66 763.272
pas d'annotation ah c'est bon ça 858.139 859.118
pas d'annotation oui 1074.61 1075.505
pas d'annotation Thierry 1080.287 1080.548
pas d'annotation euh Skrela va passer au centre 1088.728 1090.002
pas d'annotation il dirige 1153.387 1153.955
pas d'annotation heureusement 1207.5 1208.273
pas d'annotation alors 1439.568 1440.665
pas d'annotation oui 1439.568 1440.665
pas d'annotation oh 1478.032 1478.245
pas d'annotation ça 1478.032 1478.245
pas d'annotation oh 1488.636 1488.855
[1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 3, 1, 1, 2, 2, 1, 2, 4, 1, 1, 2, 3, 1, 1, 1, 1, 2, 2, 1, 5, 1, 1, 6, 3, 2, 1, 1, 3, 3]
42.529 argentin 0 1
96.868 David Skrela 1 1
100.106 Thierry 2 1
216.835 au ra

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.


34 1
1430.599 Pichot 35 5
1447.939 et là 36 1
1487.467 regardez 37 1
1502.257 Pichot 38 6
1508.11 Pichot encore 39 3
1606.078 Thierry 40 2
1618.172 au centre 41 1
pas d'annotation ouais ouais 4880.975 4881.937
pas d'annotation lui c'est une euh 4927.966 4928.711
pas d'annotation ça se 4937.836 4938.233
pas d'annotation oui oui 4938.233 4938.724
pas d'annotation et là 5016.395 5016.915
pas d'annotation je dois dire 5016.915 5017.337
pas d'annotation que le ralenti 5017.337 5018.164
pas d'annotation c'est 5041.862 5042.037
pas d'annotation elle 5148.872 5149.166
pas d'annotation bravo 5149.166 5149.617
pas d'annotation est impor(tante) 5149.166 5149.617
pas d'annotation avec 5158.054 5158.39
pas d'annotation dans le 5172.071 5172.416
pas d'annotation ah 5172.071 5172.416
pas d'annotation sur ses 5176.069 5176.689
pas d'annotation ah ils cassent le rythme il faut ils veulent jouer à leur à leur niveau c'est un jeu tranquille pépère dès qu'on met du rythme et ben ils sont 5204.46 5210.159


In [1454]:
partiesDF["CM"]

,start,end,texte,discours,émotion,excitation,début,fin
0,4793.887,4795.443,oh il avait très bien joué Chabal il est allé,point de vue,satisfaction,2,5848.49,5850.06
1,4795.687,4799.498,s'engager où tout le monde s'attendait à ce qu...,explication,,2,5850.3,5854.14
2,4799.818,4802.536,mais là l'action le voulait et il a superbemen...,point de vue,satisfaction,2,5854.45,5857.18
3,4803.027,4808.130,contact passer les bras tourner dans le mouvem...,description,,2,5857.66,5862.78
4,4808.468,4809.780,donné par Chabal peut-être,point de vue,,2,5863.1,5864.41
5,4810.118,4810.897,qui l'a fait commettre un en-avant,point de vue,,2,5865.12,5865.9
6,4812.030,4812.499,alors que c'est,explication,,1,5866.66,5867.14
7,4812.820,4816.387,alors que Betsen a récupéré le capitanat suite...,explication,,1,5867.45,5871.02
8,4816.387,4816.639,ouais,régulation,,1,5871.02,5871.26
9,4817.001,4819.460,et surtout David Skrela qui se plaint de son a...,description,,1,5871.62,5874.1


## Partie MP

In [733]:
xmlTRSMP=etree.parse(fAnnotationsMP,parser)

In [734]:
listeToursMP=[]
for ligne in xmlTRSMP.xpath("//Turn"):
    debut=float(ligne.attrib["startTime"])
    if debut<=maxTimeMP:
        fin=float(ligne.attrib["endTime"])
        lTextes=[e.strip() for e in list(ligne.itertext()) if e.strip()!=""]
        lTour=[]
        lDiscours=[]
        lEmotions=[]
        lExcitation=[]
        for l in lTextes:
            splitL=[e.strip() for e in re.split(ur"\[([^\]]*)\]",l) if e!=""]
            lenSplitL=len(splitL)
            if lenSplitL%2!=0:
                print "pas d'annotation", l, debut, fin
                lTour.append(l.strip())
            else:
                for tour,crochet in zip(*(iter(splitL),) * 2):
                    discours=""
                    emotion=""
                    excitation=""
                    lTour.append(tour.strip())
                    annotations=crochet.split(";")
                    if len(annotations)==3:
                        discours,emotion,excitation=annotations
                    elif len(annotations)==2:
                        discours,excitation=annotations
                    else:
                        print "problème d'annotation", annotations, debut, fin
                    lDiscours.append(discours)
                    lEmotions.append(emotion)
                    lExcitation.append(excitation)
        if lTour: listeToursMP.append((debut,fin,jList(lTour,sep=" "),jList(lDiscours),jList(lEmotions),jList(lExcitation)))                    

pas d'annotation oh hé bien 171.469 172.067
pas d'annotation euh Bernard Laporte euh 267.258 268.701
pas d'annotation oui 671.256 673.124
pas d'annotation ah ouais 694.107 694.65
pas d'annotation c'est un 698.763 699.06
pas d'annotation ouais on 712.739 713.255
pas d'annotation il est pris 761.66 763.272
pas d'annotation ah c'est bon ça 858.139 859.118
pas d'annotation oui 1074.61 1075.505
pas d'annotation Thierry 1080.287 1080.548
pas d'annotation euh Skrela va passer au centre 1088.728 1090.002
pas d'annotation il dirige 1153.387 1153.955
pas d'annotation heureusement 1207.5 1208.273
pas d'annotation alors 1439.568 1440.665
pas d'annotation oui 1439.568 1440.665
pas d'annotation oh 1478.032 1478.245
pas d'annotation ça 1478.032 1478.245
pas d'annotation oh 1488.636 1488.855
pas d'annotation ça me permet d'ailleurs Thierry de rappeler que 1620.449 1622.327
pas d'annotation voilà pourquoi on joue à vingt-deux au rugby aussi 1622.618 1624.537
pas d'annotation parce que grâce au saigneme

In [735]:
jList(["direct",u"il en avait le droit il \xe9tait \xe0 l'int\xe9rieur de ses vingt-deux m\xe8tres"],sep=" ")

u"direct il en avait le droit il \xe9tait \xe0 l'int\xe9rieur de ses vingt-deux m\xe8tres"

In [736]:
colAnnotations=["start","end","texte","discours","émotion","excitation"]
dfMP=pd.DataFrame.from_records(listeToursMP,columns=colAnnotations)

In [737]:
dfMP

,start,end,texte,discours,émotion,excitation
0,33.121,36.835,après avoir demandé si tout le monde était prê...,narration,,2
1,37.086,39.447,est donné par David Skrela.,narration,,2
2,40.492,41.904,allez de suite dans le camp euh,narration,,2
3,42.529,43.080,argentin,narration,,2
4,43.213,44.769,avec un départ de Roncero,narration suspensive,,2
5,45.468,46.684,Pichot a appelé le ballon,narration,,2
6,47.456,48.706,pour Hernandez,narration suspensive,,2
7,49.388,50.910,avec le pied droit,narration suspensive,,2
8,51.473,53.200,pour trouver une très jolie touche,narration,admiration,2
9,53.894,57.564,direct il en avait le droit il était à l'intér...,"narration,explication",,1


In [738]:
texteRef=[s.strip() for s in df[(df["texte"].notnull()) & (df["début"]<1619)]["texte"].tolist()]
texteRef=[l for l in texteRef if not l.startswith("spk")]
texteAnnot=[s.strip()for s in dfMP["texte"].tolist() if s!=""]
texteRef,texteAnnot

([u"apr\xe8s avoir demand\xe9 si tout le monde \xe9tait pr\xeat, le coup d'envoi de la coupe du monde deux mille sept",
  u'est donn\xe9 par David Skrela.',
  u'allez de suite dans le camp euh',
  u'argentin',
  u'avec un d\xe9part de Roncero',
  u'Pichot a appel\xe9 le ballon',
  u'pour Hernandez',
  u'avec le pied droit',
  u'pour trouver une tr\xe8s jolie touche',
  u"direct il en avait le droit il \xe9tait \xe0 l'int\xe9rieur de ses vingt-deux m\xe8tres",
  u'dans ces cas-l\xe0',
  u'on peut trouver une touche',
  u'directement',
  u'alors la touche',
  u'\xe7a \xe7a sera une des clefs de cette rencontre',
  u'le bloc fran\xe7ais',
  u'contre le bloc argentin',
  u'lancer',
  u"pour le capitaine de l'\xe9quipe de France Rapha\xebl Iba\xf1ez",
  u'qui a donc le choix de la combinaison',
  u'ah un ballon bien pris ah quoique rel\xe2ch\xe9',
  u'et \xe7a profite \xe0 Pichot',
  u'un petit coup de pied par-dessus',
  u'C\xe9dric Heymans en couverture il va devoir',
  u'oui avec la cuil

In [739]:
dfMP["début"]=""
dfMP["fin"]=""
for t,ref,annot in [(i == j, i,j) for i, j in zip(texteRef, texteAnnot)]:
    if t:
        debut=df.loc[df["texte"]==ref,"début"].values
        fin=df.loc[df["texte"]==ref,"fin"].values
        if len(debut)==1:
            dfMP.loc[dfMP["texte"]==ref,"début"]=debut[0]
        if len(fin)==1:
            dfMP.loc[dfMP["texte"]==ref,"fin"]=fin[0]


In [740]:
ixChoix=[
    1,1,1,1,1,1,2,1,1,2,
    1,1,1,1,2,2,2,3,1,1,
    2,2,1,2,4,1,1,2,3,1,
    1,1,1,2,2,1,5,1,1,6,
    3,2,1,1,3,3
    ]

In [741]:
numChoix=0
for ix,row in dfMP[dfMP["début"]==""].iterrows():
    ref=row["texte"]
    start=row["start"]
    tDebut=df.loc[df["texte"]==ref,"début"].values
    xDebut=df[df["texte"]==ref].index
    if tDebut!=[]:
        ixSelect=ixChoix[numChoix]
        print start, ref, numChoix, ixSelect
        debut=df.iloc[xDebut[ixSelect-1]]["début"]
        fin=df.iloc[xDebut[ixSelect-1]]["fin"]
        if abs(start-debut)>1:
            print "erreur probable",numChoix
            print ixSelect,debut,fin, "(%f)"%(start-debut)
            print tDebut
        dfMP.iloc[ix,dfMP.columns.get_loc('début')]=debut
        dfMP.iloc[ix,dfMP.columns.get_loc('fin')]=fin
        numChoix+=1

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  import sys


42.529 argentin 0 1
96.868 David Skrela 1 1
100.106 Thierry 2 1
216.835 au ras 3 1
244.419 au sol 4 1
300.957 face à l'expérience 5 1
344.285 David Skrela 6 2
362.355 Fabrice 7 1
467.19 Mignoni 8 1
486.357 Mignoni 9 2
490.67 Heymans 10 1
497.301 Skrela 11 1
520.995 Pichot 12 1
521.516 voilà 13 1
522.499 Pichot 14 2
544.992 argentin 15 2
554.67 Fabrice 16 2
641.664 Pichot 17 3
649.197 Pichot encore 18 1
693.756 Pelous 19 1
735.397 Pichot encore 20 2
859.118 au ras 21 2
904.799 français 22 1
1132.632 Pelous 23 2
1190.639 Pichot 24 4
1200.447 Hernandez 25 1
1207.5 heureusement 26 1
1248.564 voilà 27 2
1277.982 Mignoni 28 3
1279.303 pour Skrela 29 1
1349.158 qui 30 1
1382.948 de l'Argentine 31 1
1391.175 Ledesma 32 1
1394.535 Hernandez 33 2
1398.405 Heymans 34 2
1400.395 ouais 35 1
1430.599 Pichot 36 5
1447.939 et là 37 1
1487.467 regardez 38 1
1502.257 Pichot 39 6
1508.11 Pichot encore 40 3
1606.078 Thierry 41 2
1618.172 au centre 42 1
1639.312 et Hernandez 43 1
1646.96 Heymans 44 3


In [742]:
dfMP[dfMP["début"]==""]

,start,end,texte,discours,émotion,excitation,début,fin
662,1620.449,1622.327,ça me permet d'ailleurs Thierry de rappeler que,,,,,
663,1622.618,1624.537,voilà pourquoi on joue à vingt-deux au rugby a...,,,,,
664,1624.739,1628.043,parce que grâce au saignement en tout cas il f...,,,,,
665,1628.388,1630.774,donc mentalement il faut être concentré quand ...,,,,,
666,1631.022,1631.484,c'est plus dur,,,,,
667,1631.694,1632.503,parce qu'on y est pas dedans,,,,,
668,1632.854,1633.653,il faut rentrer et resortir,,,,,
669,1634.043,1636.327,Fernandez Lobbe a très bien pris ce ballon,,,,,
670,1636.561,1637.535,sur le renvoi français,,,,,
672,1642.993,1645.873,ne trouvera pas la touche il est tombé sur Har...,,,,,


In [743]:
with open("dfMP.pkl","wb") as outFile:
    pickle.dump(dfMP[:662], outFile, pickle.HIGHEST_PROTOCOL)

In [744]:
with open("dfMP.pkl","rb") as inFile:
    dfMP=pickle.load(inFile)

In [751]:
dfMP[dfMP["début"]==""]

,start,end,texte,discours,émotion,excitation,début,fin


## Partie CF

In [808]:
xmlTRSCF=etree.parse(fAnnotationsCF,parser)

In [809]:
listeToursCF=[]
for ligne in xmlTRSCF.xpath("//Turn"):
    debut=float(ligne.attrib["startTime"])
    if minTimeCF<=debut<=maxTimeCF:
        fin=float(ligne.attrib["endTime"])
        lTextes=[e.strip() for e in list(ligne.itertext()) if e.strip()!=""]
        lTour=[]
        lDiscours=[]
        lEmotions=[]
        lExcitation=[]
        for l in lTextes:
            splitL=[e.strip() for e in re.split(ur"\[([^\]]*)\]",l) if e!=""]
            lenSplitL=len(splitL)
            if lenSplitL%2!=0:
                print "pas d'annotation", l, debut, fin
                lTour.append(l.strip())
            else:
                for tour,crochet in zip(*(iter(splitL),) * 2):
                    discours=""
                    emotion=""
                    excitation=""
                    lTour.append(tour.strip())
                    annotations=crochet.split(";")
                    if len(annotations)==3:
                        discours,emotion,excitation=annotations
                    elif len(annotations)==2:
                        discours,excitation=annotations
                    else:
                        print "problème d'annotation", annotations, debut, fin
                    lDiscours.append(discours)
                    lEmotions.append(emotion)
                    lExcitation.append(excitation)
        if lTour: listeToursCF.append((debut,fin,jList(lTour,sep=" "),jList(lDiscours),jList(lEmotions),jList(lExcitation)))                    

pas d'annotation on se barre de ah ça y est hein ça y est mon cher Thierry 3474.754 3477.864
pas d'annotation hé Rémi là ça 3493.546 3498.155
pas d'annotation ou alors un problème de couleurs entre le bleu clair et 3507.882 3511.509
pas d'annotation bleu 3507.882 3511.509
pas d'annotation regardez le un il est en position de centre là argentin 3551.229 3554.516
pas d'annotation regardez 3551.229 3554.516
pas d'annotation et voilà il appelle d'ailleurs 3554.952 3558.492
pas d'annotation avec ce ballon 3719.14 3722.302
pas d'annotation là 3719.14 3722.302
pas d'annotation ah il y a des sécateurs hein vous l'avez dit 3753.417 3755.611
pas d'annotation ça 3753.417 3755.611
pas d'annotation regardez 3753.417 3755.611
pas d'annotation Thierry 3871.687 3872.135
pas d'annotation l'homme 3874.398 3874.931
pas d'annotation ouais 4013.905 4017.32
pas d'annotation hein 4020.501 4020.974
pas d'annotation oui 4162.723 4162.897
pas d'annotation espoir et même confiance à l'équipe de France 4172.664 4

In [810]:
dfCF=pd.DataFrame.from_records(listeToursCF,columns=colAnnotations)

In [811]:
texteRef=[s.strip() for s in df[(df["texte"].notnull()) & (df["début"]<=maxTimeCF+decMiTemps) & (df["début"]>=minTimeCF+decMiTemps)]["texte"].tolist()]
texteRef=[l for l in texteRef if not l.startswith("spk")]
texteAnnot=[s.strip()for s in dfCF["texte"].tolist() if s!="" and s!=u"qui l'a fait commettre un en-avant"]
#texteRef,
texteAnnot

[u'et vous revenez au stade de France au moment o\xf9 le quinze de France revient',
 'sur la pelouse',
 u'du stade de France une \xe9quipe de France men\xe9e dix-sept \xe0 neuf',
 "par l'Argentine on imagine Thierry Lacroix",
 u'que dans le vestiaire \xe7a a \xe9t\xe9 s\xe9ance disco',
 'vous voyez ce que je veux dire',
 'autour de Bernard Laporte hein',
 u"ouais surtout quand on voit le la t\xeate qu'il fait mais il a raison",
 "mais c'est c'est",
 u"\xe9coutez mon cher Thierry aujourd'hui il faut \xeatre clair premi\xe8re mi-temps on leur donne la France donne \xe0 l'Argentine tous ses points",
 u"c'est tous nos ballons \xe0 nous qu'on leur on a",
 'et on les tombe on leur donne ils interceptent ils vont marquer un essai',
 u'h\xe9 si on les garde ces ballons',
 "c'est vingt points dans l'autre dans l'autre sens",
 u"c'est pour \xe7a que la deuxi\xe8me mi-temps je suis persuad\xe9",
 'que non seulement ils on pris un soufflon',
 'par Bernard Laporte',
 u'mais en plus de \xe7a h\xe9',

In [822]:
dfCF["début"]=""
dfCF["fin"]=""
for t,ref,annot in [(i == j, i,j) for i, j in zip(texteRef, texteAnnot)]:
    if t:
        debut=df.loc[df["texte"]==ref,"début"].values
        fin=df.loc[df["texte"]==ref,"fin"].values
        if len(debut)==1:
            dfCF.loc[dfCF["texte"]==ref,"début"]=debut[0]
        if len(fin)==1:
            dfCF.loc[dfCF["texte"]==ref,"fin"]=fin[0]


In [823]:
dfCF[dfCF["début"]==""]

,start,end,texte,discours,émotion,excitation,début,fin
52,3389.224,3389.603,allez,narration suspensive,espoir,2,,
54,3391.248,3392.007,au point de chute,narration,satisfaction,2,,
72,3452.421,3453.144,avec euh,narration suspensive,,1,,
74,3458.927,3459.899,Mignoni encore,narration suspensive,,2,,
86,3486.254,3486.432,à,narration,,2,,
109,3536.836,3537.486,Mignoni,narration,,2,,
110,3537.766,3538.433,Skrela,narration,,2,,
120,3564.560,3565.619,avec Skrela,narration,,2,,
129,3585.989,3586.989,avec Dominici,narration,,2,,
141,3621.790,3621.940,ouais,narration,,2,,


In [824]:
numTour=625
decalagesTemp=[]
for numTour in range(622,627):
    decalagesTemp.append(dfCF.iloc[numTour]["début"]-dfCF.iloc[numTour]["start"])
    decalagesTemp.append(dfCF.iloc[numTour]["fin"]-dfCF.iloc[numTour]["end"])
decTemp=np.mean(decalagesTemp)

In [825]:
ixInsertion=dfCF[dfCF["texte"]==u"qui l'a fait commettre un en-avant"].index
dfCF.loc[ixInsertion,"début"]=dfCF.loc[ixInsertion,"start"]+decTemp
dfCF.loc[ixInsertion,"fin"]=dfCF.loc[ixInsertion,"end"]+decTemp

In [826]:
ixChoix=[2,2,1,2,1,
         9,4,2,1,8,
         9,2,1,10,1,
         2,3,1,5,8,
         2,1,6,10,2,
         2,2,1,2,3,
         9,1,11,1,2,
         1,12,1,2,2,
         5,4,4,5,3,
         13,3,4,3,1,
         2,1,2,2,2,
         1,14,4,6]

In [827]:
numChoix=0
for ix,row in dfCF[dfCF["début"]==""].iterrows():
    ref=row["texte"]
    start=row["start"]+decMiTemps
    tDebut=df.loc[df["texte"]==ref,"début"].values
    xDebut=df[df["texte"]==ref].index
    if tDebut!=[]:
        ixSelect=ixChoix[numChoix]
        print start, ref, numChoix, ixSelect
        debut=df.iloc[xDebut[ixSelect-1]]["début"]
        fin=df.iloc[xDebut[ixSelect-1]]["fin"]
        if abs(start-debut)>1:
            print "erreur probable",numChoix
            print ixSelect,debut,fin, "(%f)"%(start-debut)
            print tDebut
        dfCF.ix[ix,"début"]=debut
        dfCF.ix[ix,"fin"]=fin
        numChoix+=1

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  import sys
/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


4444.224 allez 0 2
4446.248 au point de chute 1 2
4507.421 avec euh 2 1
4513.927 Mignoni encore 3 2
4541.254 à 4 1
4591.836 Mignoni 5 9
4592.766 Skrela 6 4
4619.56 avec Skrela 7 2
4640.989 avec Dominici 8 1
4676.79 ouais 9 8
4685.924 ouais 10 9
4697.425 dans la moitié de terrain de l'Argentine 11 2
4735.757 c'est parti 12 1
4839.748 Mignoni 13 10
4873.608 il faut rester debout 14 1
4874.713 il faut rester debout 15 2
4922.047 argentin 16 3
4946.481 et 17 1
5012.682 Hernandez 18 5
5027.133 Pichot 19 8
5029.247 et Hernandez 20 2
5032.109 ballon récupéré par 21 1
5041.798 Hernandez 22 6
5078.323 ouais 23 10
5100.071 au centre 24 2
5179.716 tout à fait 25 2
5189.063 avec euh 26 2
5216.599 bien 27 1
5217.723 oui 28 2
5251.319 Ledesma 29 3
5257.57 Pichot 30 9
5268.313 il est parti 31 1
5356.01 ouais 32 11
5406.331 ah oui 33 1
5413.092 c'est parti 34 2
5429.339 oui oui 35 1
5461.982 ouais 36 12
5490.993 oh la 37 1
5491.368 non 38 2
5491.707 oh la 39 2
5539.025 Skrela 40 5
5558.027 Heymans 41 

In [829]:
dfCF[dfCF["début"]!=""]

,start,end,texte,discours,émotion,excitation,début,fin
0,3257.608,3260.947,et vous revenez au stade de France au moment o...,narration suspensive,,2,4312.22,4315.58
1,3261.257,3262.315,sur la pelouse,narration,,2,4315.9,4316.94
2,3262.888,3265.924,du stade de France une équipe de France menée ...,"narration,description",,"1,2",4317.5,4320.54
3,3266.422,3269.089,par l'Argentine on imagine Thierry Lacroix,"description,point de vue",,"1,2",4321.06,4323.7
4,3269.438,3271.699,que dans le vestiaire ça a été séance disco,narration,,2,4324.06,4326.34
5,3271.699,3273.451,vous voyez ce que je veux dire,point de vue,,2,4326.34,4328.1
6,3273.451,3274.798,autour de Bernard Laporte hein,narration,,1,4328.1,4329.42
7,3274.798,3276.849,ouais surtout quand on voit le la tête qu'il f...,narration,,2,4329.42,4331.46
8,3277.161,3277.637,mais c'est c'est,narration suspensive,,2,4331.78,4332.26
9,3278.420,3283.865,écoutez mon cher Thierry aujourd'hui il faut ê...,narration,,2,4333.06,4338.5


In [830]:
with open("dfCF.pkl","wb") as outFile:
    pickle.dump(dfCF, outFile, pickle.HIGHEST_PROTOCOL)

In [744]:
with open("dfCF.pkl","rb") as inFile:
    dfCF=pickle.load(inFile)

In [831]:
dfCF[dfCF["début"]==""]

,start,end,texte,discours,émotion,excitation,début,fin


## Partie CM

In [1071]:
xmlTRSCM=etree.parse(fAnnotationsCM,parser)

In [1072]:
listeToursCM=[]
for ligne in xmlTRSCM.xpath("//Turn"):
    debut=float(ligne.attrib["startTime"])
    if minTimeCM<=debut<=maxTimeCM:
        fin=float(ligne.attrib["endTime"])
        lTextes=[e.strip() for e in list(ligne.itertext()) if e.strip()!=""]
        lTour=[]
        lDiscours=[]
        lEmotions=[]
        lExcitation=[]
        for l in lTextes:
            splitL=[e.strip() for e in re.split(ur"\[([^\]]*)\]",l) if e!=""]
            lenSplitL=len(splitL)
            if lenSplitL%2!=0:
                print "pas d'annotation", l, debut, fin
                lTour.append(l.strip())
            else:
                for tour,crochet in zip(*(iter(splitL),) * 2):
                    discours=""
                    emotion=""
                    excitation=""
                    lTour.append(tour.strip())
                    annotations=crochet.split(";")
                    if len(annotations)==3:
                        discours,emotion,excitation=annotations
                    elif len(annotations)==2:
                        discours,excitation=annotations
                    else:
                        print "problème d'annotation", annotations, debut, fin
                    lDiscours.append(discours)
                    lEmotions.append(emotion)
                    lExcitation.append(excitation)
        if lTour: listeToursCM.append((debut,fin,jList(lTour,sep=" "),jList(lDiscours),jList(lEmotions),jList(lExcitation)))                    

pas d'annotation ouais ouais 4880.975 4881.937
pas d'annotation lui c'est une euh 4927.966 4928.711
pas d'annotation ça se 4937.836 4938.233
pas d'annotation oui oui 4938.233 4938.724
pas d'annotation et là 5016.395 5016.915
pas d'annotation je dois dire 5016.915 5017.337
pas d'annotation que le ralenti 5017.337 5018.164
pas d'annotation c'est 5041.862 5042.037
pas d'annotation elle 5148.872 5149.166
pas d'annotation bravo 5149.166 5149.617
pas d'annotation est impor(tante) 5149.166 5149.617
pas d'annotation avec 5158.054 5158.39
pas d'annotation dans le 5172.071 5172.416
pas d'annotation ah 5172.071 5172.416
pas d'annotation sur ses 5176.069 5176.689
pas d'annotation ah ils cassent le rythme il faut ils veulent jouer à leur à leur niveau c'est un jeu tranquille pépère dès qu'on met du rythme et ben ils sont 5204.46 5210.159
pas d'annotation oui mais bon il faut les 5213.114 5214.52
pas d'annotation avec euh 5269.89 5270.737
pas d'annotation on est à la 5447.389 5448.305
pas d'annotati

In [1073]:
dfCM=pd.DataFrame.from_records(listeToursCM,columns=colAnnotations)

In [1122]:
listeInsertionCM=[u"qui l'a fait commettre un en-avant",u"surclassés",u"ballons"]
texteRef=[s.strip() for s in df[(df["texte"].notnull()) & (df["début"]<=maxTimeCM+decMiTemps) & (df["début"]>=minTimeCM+decMiTemps)]["texte"].tolist()]
texteRef=[l for l in texteRef if not l.startswith("spk")]
texteAnnot=[s.strip()for s in dfCM["texte"].tolist() if s!="" and s not in listeInsertionCM]
texteRef,
#texteAnnot

([u'oh il avait tr\xe8s bien jou\xe9 Chabal il est all\xe9',
  u"s'engager o\xf9 tout le monde s'attendait \xe0 ce qu'il garde la balle parce que d'habitude il ne fait pas de passe",
  u"mais l\xe0 l'action le voulait et il a superbement bien jou\xe9",
  u"contact passer les bras tourner dans le mouvement \xe0 Betsen c'est s\xfbrement la surprise de Serge Betsen d'avoir un ballon",
  u'donn\xe9 par Chabal peut-\xeatre',
  u"alors que c'est",
  u'alors que Betsen a r\xe9cup\xe9r\xe9 le capitanat suite \xe0 la sortie de Rapha\xebl Iba\xf1ez',
  u'ouais',
  u'et surtout David Skrela qui se plaint de son avant-bras',
  u"ouais puis peut-\xeatre m\xeame de ses jambes on on l'a \xe0 l'image euh Fabrice",
  u'David Skrela',
  u'ben oui les les le capitaine et le vice-capitaine \xe9tant sortis',
  u"je veux parler d'Iba\xf1ez et de Pelous",
  u"c'est Betsen qui fait partie du comit\xe9 des sages",
  u'de cette \xe9quipe de France',
  u'qui a \xe9t\xe9 euh',
  u'capitaine contre le Pays de Gall

In [1123]:
dfCM["début"]=""
dfCM["fin"]=""
for t,ref,annot in [(i == j, i,j) for i, j in zip(texteRef, texteAnnot)]:
    if t:
        debut=df.loc[df["texte"]==ref,"début"].values
        fin=df.loc[df["texte"]==ref,"fin"].values
        if len(debut)==1:
            dfCM.loc[dfCM["texte"]==ref,"début"]=debut[0]
        if len(fin)==1:
            dfCM.loc[dfCM["texte"]==ref,"fin"]=fin[0]


In [1124]:
dfCM.iloc[410:][dfCM["début"]==""]

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,start,end,texte,discours,émotion,excitation,début,fin
412,5751.300,5751.983,Pichot,narration suspensive,,1,,
413,5753.025,5754.684,Hernandez,narration suspensive,,1,,
415,5757.732,5758.445,Heymans,narration suspensive,,2,,
423,5783.733,5783.995,qui,,,,,
424,5784.343,5785.177,Hernandez,,,,,
428,5789.280,5789.539,ouais,régulation,,1,,
433,5800.037,5800.485,heureusement,point de vue,,2,,
436,5810.067,5811.324,introduction Pichot,narration suspensive,,2,,
439,5817.717,5818.885,avec Senillosa,narration suspensive,,2,,
458,5884.333,5884.633,allez,,,1,,


In [824]:
numTour=625
decalagesTemp=[]
for numTour in range(622,627):
    decalagesTemp.append(dfCM.iloc[numTour]["début"]-dfCM.iloc[numTour]["start"])
    decalagesTemp.append(dfCM.iloc[numTour]["fin"]-dfCM.iloc[numTour]["end"])
decTemp=np.mean(decalagesTemp)

In [1030]:
for insertion in listeInsertionCM:
    ixInsertion=dfCM[dfCM["texte"]==insertion].index
    dfCM.loc[ixInsertion,"début"]=dfCM.loc[ixInsertion,"start"]+decTemp
    dfCM.loc[ixInsertion,"fin"]=dfCM.loc[ixInsertion,"end"]+decTemp

In [826]:
ixChoix=[]

In [ ]:
for ix,row in dfCM[dfCM["début"]==""].iterrows():
    ref=row["texte"]
    start=row["start"]
    debut=df.loc[df["texte"]==ref,"début"].values
    xDebut=df[df["texte"]==ref].index
    if debut!=[]:
        print start+decMiTemps, ref
        print debut
        print xDebut
        ixSelect=raw_input("Select index\n")
        ixChoix.append(ixSelect)
        if ixSelect:
            debut=df.iloc[xDebut[int(ixSelect)-1]]["début"]
            fin=df.iloc[xDebut[int(ixSelect)-1]]["fin"]    
            print debut,fin
            dfCM.iloc[ix,dfCM.columns.get_loc('début')]=debut
            dfCM.iloc[ix,dfCM.columns.get_loc('fin')]=fin
        print

In [1101]:
dfCM[dfCM["début"]!=""]

,start,end,texte,discours,émotion,excitation,début,fin
0,4793.887,4795.443,oh il avait très bien joué Chabal il est allé,point de vue,satisfaction,2,5848.49,5850.06
1,4795.687,4799.498,s'engager où tout le monde s'attendait à ce qu...,explication,,2,5850.3,5854.14
2,4799.818,4802.536,mais là l'action le voulait et il a superbemen...,point de vue,satisfaction,2,5854.45,5857.18
3,4803.027,4808.130,contact passer les bras tourner dans le mouvem...,description,,2,5857.66,5862.78
4,4808.468,4809.780,donné par Chabal peut-être,point de vue,,2,5863.1,5864.41
6,4812.030,4812.499,alors que c'est,explication,,1,5866.66,5867.14
7,4812.820,4816.387,alors que Betsen a récupéré le capitanat suite...,explication,,1,5867.45,5871.02
9,4817.001,4819.460,et surtout David Skrela qui se plaint de son a...,description,,1,5871.62,5874.1
10,4820.165,4823.902,ouais puis peut-être même de ses jambes on on ...,description,,2,5874.78,5878.54
12,4827.162,4829.946,ben oui les les le capitaine et le vice-capita...,explication,,1,5881.78,5884.58


In [830]:
with open("dfCM.pkl","wb") as outFile:
    pickle.dump(dfCM, outFile, pickle.HIGHEST_PROTOCOL)

In [744]:
with open("dfCM.pkl","rb") as inFile:
    dfCM=pickle.load(inFile)

In [831]:
dfCM[dfCM["début"]==""]

,start,end,texte,discours,émotion,excitation,début,fin


## Gestion des parties

In [820]:
print ",".join([str(i) for i in ixChoix])

2,2,1,2,1,9,4,2,1,8,9,2,1,10,1,2,3,1,5,8,2,1,6,10,2,2,2,1,2,3,9,1,11,1,2,1,12,1,2,2,5,4,4,5,3,13,3,4,3,1,2,1,2,2,2,1,14,4,6


# Marquage des noms propres

In [832]:
fJoueurs="NomsJoueurs.txt"
with codecs.open(fJoueurs,"r",encoding="utf8") as inFile:
    lJoueurs="".join(inFile.readlines())

tEquipes=re.split(u"\n(?=.*:)",lJoueurs)

In [833]:
equipes={}
for tEquipe in tEquipes[1:]:
    print tEquipe
    lEquipe=tEquipe.split("\n")
    nomEquipe=lEquipe[0].split(":")[0].strip()
    equipes[nomEquipe]={}
    equipe=equipes[nomEquipe]
    for ligne in lEquipe[1:]:
        m=re.match(ur"(.*)\t(.*)",ligne)
        if m:
            print "poste",m.group(1)
            print "joueurs",m.group(2).split(",")
            poste=m.group(1).strip()
            joueurs=[j.strip() for j in m.group(2).split(",") if j.strip()!=""]
            if joueurs:
                equipe[poste]=joueurs
    

Angleterre : 

Entraîneur	Brian Ashton
Avants	
	Pilier	Perry Freshwater, Andrew Sheridan, Matt Stevens, Phil Vickery
	Talonneur	George Chuter, Lee Mears, Mark Regan
	Deuxième ligne	Steve Borthwick, Ben Kay, Simon Shaw
	Troisième ligne aile	Lewis Moody, Tom Rees, Joe Worsley
	Troisième ligne centre	Martin Corry, Lawrence Dallaglio, Nick Easter
Arrières	
	Demi de mêlée	Andy Gomarsall, Shaun Perry, Peter Richards
	Demi d'ouverture	Jonny Wilkinson, Olly Barkley
	Centre	Mike Catt, Andy Farrell, Dan Hipkiss, Jamie Noon, Mathew Tait, Toby Flood
	Ailier	Jason Robinson, Paul Sackey
	Arrière	Mark Cueto, Josh Lewsey



poste Entraîneur
joueurs [u'Brian Ashton\r']
poste Avants
joueurs [u'\r']
poste 	Pilier
joueurs [u'Perry Freshwater', u' Andrew Sheridan', u' Matt Stevens', u' Phil Vickery\r']
poste 	Talonneur
joueurs [u'George Chuter', u' Lee Mears', u' Mark Regan\r']
poste 	Deuxième ligne
joueurs [u'Steve Borthwick', u' Ben Kay', u' Simon Shaw\r']
poste 	Troisième ligne aile
joueurs [u'Lewis Moo

In [834]:
def getJoueurs(pays):
    result=[]
    for k,v in equipes[pays].iteritems():
        result.extend(v)
        result=[j.replace("(cap.)","").strip() for j in result]
    return result

In [835]:
joueursFr=getJoueurs("France")
joueursAr=getJoueurs("Argentine")

In [836]:
def ajoutNP(lSet,np):
    decNP=unicode(unidecode.unidecode(np))
    lSet.add(np)
    if np!=decNP:
        lSet.add(decNP)

In [1489]:
def faireRegexJoueurs(joueurs):
    listeTemp=set()
    for j in joueurs:
        listeTemp.add(j)
        listeTemp.add(unidecode.unidecode(j))
    nomsComplets="|".join(sorted(listeTemp))
    prenoms=set()
    noms=set()
    for j in joueurs:
        prenom,nom=j.split(" ",1)
        ajoutNP(prenoms,prenom)
        ajoutNP(noms,nom)
        if " " in nom:
            nomParts=nom.split(" ")
            for n in nomParts:
                if len(n)>2:
                    ajoutNP(noms,n)
#    prenomsSeuls="|".join(sorted(prenoms|noms,key=lambda x:len(x),reverse=True))
#    nomsSeuls="|".join(sorted(noms,key=lambda x:len(x),reverse=True))
    nomsSepares="|".join(sorted(prenoms|noms,key=lambda x:len(x),reverse=True))
    return nomsComplets,nomsSepares

In [1490]:
nomsCompletsFr,nomsSeparesFr=faireRegexJoueurs(joueursFr)
coordJoueursFr="%s|%s"%(nomsCompletsFr,nomsSeparesFr)
nomsCompletsAr,nomsSeparesAr=faireRegexJoueurs(joueursAr)
coordJoueursAr="%s|%s"%(nomsCompletsAr,nomsSeparesAr)
regexJoueurs=u"(%s|%s|%s|%s)"%(nomsCompletsFr,nomsCompletsAr,nomsSeparesFr,nomsSeparesAr)

In [1491]:
print regexJoueurs

(Aurelien Rougerie|Aurélien Rougerie|Bernard Laporte|Cedric Heymans|Christophe Dominici|Clement Poitrenaud|Clément Poitrenaud|Cédric Heymans|Damien Traille|David Marty|David Skrela|Dimitri Szarzewski|Fabien Pelous|Frederic Michalak|Frédéric Michalak|Imanol Harinordoquy|Jean-Baptiste Elissalde|Jean-Baptiste Poux|Jean-Baptiste Élissalde|Jerome Thion|Julien Bonnaire|Jérôme Thion|Lionel Beauxis|Lionel Nallet|Nicolas Mas|Olivier Milloud|Pierre Mignoni|Pieter de Villiers|Raphael Ibanez|Raphaël Ibañez|Remy Martin|Rémy Martin|Sebastien Bruno|Sebastien Chabal|Serge Betsen|Sébastien Bruno|Sébastien Chabal|Thierry Dusautoir|Vincent Clerc|Yannick Jauzion|Yannick Nyanga|Agustin Pichot|Agustín Pichot|Alberto Vernet Basualdo|Esteban Lozada|Federico Serra|Federico Todeschini|Felipe Contepomi|Gonzalo Longo|Gonzalo Tiesi|Hernan Senillosa|Hernán Senillosa|Horacio Agulla|Ignacio Corleto|Ignacio Fernandez Lobbe|Ignacio Fernández Lobbe|Jose Maria Nunez Piossek|José María Núñez Piossek|Juan Manuel Leguizamon

In [1495]:
def identifierJoueurs(liste):
    identites=set()
    for n in liste:
        for j in joueursFr:
            if unidecode.unidecode(n) in unidecode.unidecode(j): 
                idFr=j+"=Fr"
                identites.add(idFr)
        for j in joueursAr:
            if unidecode.unidecode(n) in unidecode.unidecode(j):
                idAr=j+"=Ar"
                identites.add(idAr)
    return ";".join(identites)


In [1496]:

m=re.findall(regexJoueurs,u'David Skrela')
if m:
    print m
    tourNP=identifierJoueurs(m)
    print tourNP

[u'David Skrela']
David Skrela=Fr


## Joueurs partie MP

In [843]:
for ix,row in dfMP.iterrows():
    m=re.findall(regexJoueurs,row["texte"])
    if m:
        tourNP=identifierJoueurs(m)
        print tourNP
        dfMP.ix[ix,"noms"]=tourNP

David Marty=Fr=David;David Skrela=Fr=David,Skrela
Rodrigo Roncero=Ar=Roncero
Agustín Pichot=Ar=Pichot
Juan Martín Hernández=Ar=Hernandez
Raphaël Ibañez=Fr=Raphaël,Ibañez
Agustín Pichot=Ar=Pichot
Cédric Heymans=Fr=Cédric,Heymans
Agustín Pichot=Ar=Pichot
Cédric Heymans=Fr=Heymans
David Marty=Fr=David;David Skrela=Fr=David,Skrela
Thierry Dusautoir=Fr=Thierry
Agustín Pichot=Ar=Agustin,Pichot
Cédric Heymans=Fr=Heymans
Mario Ledesma=Ar=Ledesma
Mario Ledesma=Ar=Mario,Ledesma,Mario
Mario Ledesma=Ar=Ledesma
Juan Martín Hernández=Ar=Hernandez
Cédric Heymans=Fr=Cédric,Heymans
Martín Durand=Ar=Martin;Martín Schusterman=Ar=Martin;Juan Martín Hernández=Ar=Juan,Martin,Hernandez;Martín Scelzo=Ar=Martin;Juan Martín Fernández Lobbe=Ar=Juan,Martin;Juan Manuel Leguizamón=Ar=Juan;Rémy Martin=Fr=Martin
Cédric Heymans=Fr=Cédric,Heymans
Juan Martín Hernández=Ar=Hernandez
Mario Ledesma=Ar=Mario,Ledesma
Rémy Martin=Fr=Rémy,Martin;Martín Schusterman=Ar=Martin;Juan Martín Hernández=Ar=Martin;Martín Scelzo=Ar=Mart

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  """
/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  # This is added back by InteractiveShellApp.init_path()


Agustín Pichot=Ar=Pichot
Agustín Pichot=Ar=Pichot
Lucas Borges=Ar=Borges
Christophe Dominici=Fr=Dominici
Thierry Dusautoir=Fr=Thierry
Raphaël Ibañez=Fr=Raphaël,Ibañez
Pierre Mignoni=Fr=Pierre,Mignoni
David Marty=Fr=David;David Skrela=Fr=David,Skrela
Manuel Contepomi=Ar=Contepomi;Felipe Contepomi=Ar=Contepomi
Thierry Dusautoir=Fr=Thierry
David Skrela=Fr=Skrela
Martín Scelzo=Ar=Scelzo;Agustín Pichot=Ar=Pichot
Juan Martín Hernández=Ar=Hernandez
Lucas Ostiglia=Ar=Lucas;Lucas Borges=Ar=Lucas,Borges
Christophe Dominici=Fr=Dominici
Agustín Pichot=Ar=Pichot
Agustín Pichot=Ar=Pichot
Juan Martín Hernández=Ar=Hernandez
Pierre Mignoni=Fr=Mignoni
Cédric Heymans=Fr=Heymans
Juan Martín Hernández=Ar=Hernandez
Fabien Pelous=Fr=Fabien,Pelous
Pierre Mignoni=Fr=Mignoni
Pierre Mignoni=Fr=Mignoni
Pierre Mignoni=Fr=Mignoni
Pierre Mignoni=Fr=Mignoni
Fabien Pelous=Fr=Pelous
Juan Martín Hernández=Ar=Hernandez
Fabien Pelous=Fr=Fabien
Martín Durand=Ar=Martin;Martín Schusterman=Ar=Martin;Juan Martín Hernández=Ar=M

In [844]:
dfMP["noms"]=dfMP["noms"].fillna("")

In [845]:
dfMP[(dfMP["noms"].str.contains(";"))&(dfMP["noms"].str.contains(","))]["noms"]

1      David Marty=Fr=David;David Skrela=Fr=David,Skrela
28     David Marty=Fr=David;David Skrela=Fr=David,Skrela
55     Martín Durand=Ar=Martin;Martín Schusterman=Ar=...
68     Rémy Martin=Fr=Rémy,Martin;Martín Schusterman=...
71     Ignacio Fernández Lobbe=Ar=Fernandez,Lobbe;Jua...
99     Manuel Contepomi=Ar=Contepomi;Felipe Contepomi...
131    David Marty=Fr=David;David Skrela=Fr=David,Skrela
158    David Marty=Fr=David;David Skrela=Fr=David,Skrela
163    David Marty=Fr=David;David Skrela=Fr=David,Skrela
165    Nicolas Mas=Fr=Mas;Bernard Laporte=Fr=Bernard,...
186    David Marty=Fr=David;David Skrela=Fr=David,Skrela
190    Lucas Ostiglia=Ar=Lucas;Lucas Borges=Ar=Lucas,...
220    David Marty=Fr=David;David Skrela=Fr=David,Skrela
237    Lucas Ostiglia=Ar=Lucas;Lucas Borges=Ar=Lucas,...
289    Manuel Contepomi=Ar=Contepomi;Felipe Contepomi...
294    Rémy Martin=Fr=Rémy,Martin;Martín Schusterman=...
300    Rémy Martin=Fr=Rémy,Martin;Martín Schusterman=...
319    Martín Durand=Ar=Martin;

In [846]:
partieMP=df.merge(dfMP,how="left",on=["début","fin","texte"])

In [847]:
with open("partieMP.pkl","wb") as outFile:
    pickle.dump(partieMP, outFile, pickle.HIGHEST_PROTOCOL)

In [848]:
partieMP.to_csv(path_or_buf="partieMP.csv",sep="\t",encoding="utf8")

## Joueurs partie CF

In [849]:
for ix,row in dfCF.iterrows():
    m=re.findall(regexJoueurs,row["texte"])
    if m:
        tourNP=identifierJoueurs(m)
        print tourNP
        dfCF.ix[ix,"noms"]=tourNP

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  """
/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  # This is added back by InteractiveShellApp.init_path()
/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


Thierry Dusautoir=Fr=Thierry
Bernard Laporte=Fr=Bernard,Laporte
Thierry Dusautoir=Fr=Thierry
Bernard Laporte=Fr=Bernard,Laporte
Sébastien Chabal=Fr=Sébastien,Chabal;Sébastien Bruno=Fr=Sébastien
Bernard Laporte=Fr=Bernard,Laporte
Fabien Pelous=Fr=Pelous
Raphaël Ibañez=Fr=Ibañez
Damien Traille=Fr=Damien,Traille;Pierre Mignoni=Fr=Mignoni
David Skrela=Fr=Skrela;Bernard Laporte=Fr=Bernard,Laporte
Damien Traille=Fr=Traille
David Marty=Fr=David;David Skrela=Fr=David,Skrela
Raphaël Ibañez=Fr=Ibañez
Pierre Mignoni=Fr=Mignoni
David Skrela=Fr=Skrela
Yannick Jauzion=Fr=Jauzion
Cédric Heymans=Fr=Heymans;Christophe Dominici=Fr=Dominici
Pierre Mignoni=Fr=Mignoni
David Skrela=Fr=Skrela
Aurélien Rougerie=Fr=Rougerie
Rémy Martin=Fr=Martin;Martín Schusterman=Ar=Martin;Juan Martín Hernández=Ar=Martin;Martín Scelzo=Ar=Martin;Juan Martín Fernández Lobbe=Ar=Martin;Martín Durand=Ar=Martin
Serge Betsen=Fr=Betsen
Thierry Dusautoir=Fr=Thierry
Rémy Martin=Fr=Martin;Martín Schusterman=Ar=Martin;Juan Martín Hernánd

In [850]:
dfCF["noms"]=dfCF["noms"].fillna("")

In [851]:
dfCF[(dfCF["noms"].str.contains(";"))&(dfCF["noms"].str.contains(","))]["noms"]

24     Sébastien Chabal=Fr=Sébastien,Chabal;Sébastien...
57     Damien Traille=Fr=Damien,Traille;Pierre Mignon...
59     David Skrela=Fr=Skrela;Bernard Laporte=Fr=Bern...
62     David Marty=Fr=David;David Skrela=Fr=David,Skrela
135    David Marty=Fr=David;David Skrela=Fr=David,Skrela
230    Thierry Dusautoir=Fr=Thierry,Dusautoir;Sébasti...
231    Jean-Baptiste Poux=Fr=Jean-Baptiste,Poux;Jean-...
309    Ignacio Fernández Lobbe=Ar=Fernandez,Lobbe;Jua...
415    Yannick Jauzion=Fr=Yannick,Jauzion;Yannick Nya...
422    Jean-Baptiste Poux=Fr=Jean-Baptiste;Jean-Bapti...
462    David Marty=Fr=David;David Skrela=Fr=David,Skrela
502    Lucas Ostiglia=Ar=Lucas;Lucas Borges=Ar=Lucas,...
507    Sébastien Chabal=Fr=Sébastien,Chabal;Sébastien...
531    Rémy Martin=Fr=Rémy,Martin;Martín Schusterman=...
549    Dimitri Szarzewski=Fr=Dimitri,Szarzewski;Julie...
565    David Marty=Fr=David;David Skrela=Fr=David,Skrela
581    Martín Durand=Ar=Martin;Martín Schusterman=Ar=...
629    Raphaël Ibañez=Fr=Raphaë

In [852]:
partieCF=df.merge(dfCF,how="left",on=["début","fin","texte"])

In [853]:
with open("partieCF.pkl","wb") as outFile:
    pickle.dump(partieCF, outFile, pickle.HIGHEST_PROTOCOL)

In [854]:
partieCF.to_csv(path_or_buf="partieCF.csv",sep="\t",encoding="utf8")

# Association des joueurs aux parties

In [1510]:
def associerJoueursPartie(idAnnotations):
    dfTemp=partiesDF[idAnnotations]
    dfTemp["noms"]=np.NaN
    dfTemp["joueurs"]=np.NaN
    for ix,row in dfTemp.iterrows():
        m=re.findall(regexJoueurs,row["texte"])
        if m:
            tourNP=identifierJoueurs(m)
            dfTemp.iloc[ix,dfTemp.columns.get_loc('noms')]=u",".join(m)    
            dfTemp.iloc[ix,dfTemp.columns.get_loc('joueurs')]=tourNP

In [1517]:
dfMerges={}
for k in fAnnotations:
    associerJoueursPartie(k)
    dfMerges[k]=df.merge(partiesDF[k],how="left",on=["début","fin","texte"])
    with open("partie%s.pkl"%k,"wb") as outFile:
        pickle.dump(dfMerges[k], outFile, pickle.HIGHEST_PROTOCOL)
    dfMerges[k].to_csv(path_or_buf="partie%s.csv"%k,sep="\t",encoding="utf8")

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  """
/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  if __name__ == '__main__':


In [1516]:
dfMerges["CM"]

,RJ5.00,RC5.00,timeline,trscode,D5.00,S1D5.00,S2D5.00,S3D5.00,V5.00,S1V5.00,...,durée,plage,même,start,end,discours,émotion,excitation,noms,joueurs
0,0.0,0.0,3000.0,00:00:30.00,1.3,1.3,0,0,1.3347,1.3347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,3100.0,00:00:31.00,2,2,0,0,2.08986,2.08986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,3200.0,00:00:32.00,2.4,2.4,0,0,2.589,2.589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,3220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.5,0.5,3300.0,00:00:33.00,2.8,2.8,0,0,3.04679,3.04679,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,3349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.04,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.5,0.5,3400.0,00:00:34.00,3.1,3.1,0,0,3.3954,3.3954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,3479.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.22,0.22,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.5,0.5,3500.0,00:00:35.00,3.2,3.2,0,0,3.54374,3.54374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
